# Invite API E2E Tests

In [1]:
import requests
import random
import string
import json

In [2]:
base_url = "http://localhost:8081"

In [3]:
resp = requests.get(base_url)
assert resp.status_code == 404
assert resp.json()["status"] == 404
assert resp.json()["error"] == "Not Found"

In [4]:
resp = requests.get("%s/api/docs/swagger.json"%base_url)
assert resp.status_code == 200
assert resp.json()["swagger"] == "2.0"
assert resp.json()["paths"] != None

In [5]:
swaggerJson = resp.json()
apiEndpoints = swaggerJson["paths"].keys()
assert len(apiEndpoints)==45

In [6]:
inviteUrls = [f for f in swaggerJson["paths"].keys() if "/invite" in f]
instituteInvite = [u for u in inviteUrls if "/Institute" in u][0]
teacherInvite = [v for v in inviteUrls if "/Teacher" in v][0]
studentInvite = [w for w in inviteUrls if "/Student" in w][0]
print(instituteInvite,teacherInvite,studentInvite)

/api/v1/Institute/invite /api/v1/Teacher/invite /api/v1/Student/invite


In [7]:
def entityForTeacher(userId):
    return  {"personalDetails":{
         "userName":userId,
         "dob":"2020-12-12",
     },
      "IdentityDetails":{"type":"id","value":userId}, 
      "ContactDetails":{ "mobile":"%s"%userId,"email":"%s@example.com"%userId,
                       "address":{"street":"abc","state":"Karnataka","pincode":"560001"}}
     }

In [8]:
def entityForStudent(userId):
    return  {
        "identityDetails":{
            "fullName": "Student %s"%userId, 
            "gender": "Male"
        },
        "ContactDetails":{
            "mobile":"%s"%userId,
            "email":"%s@example.com"%userId,
            "address":{
                "street":"abc",
                "state":"Karnataka",
                "pincode":"560001"}},
        "educationDetails":[{"institute":"Some school"},
                   {"institute":"Some other school"}]
    }

In [9]:
def entityForInstitute(userId):
    return  {
        "instituteName": "%s"%userId,
        "ContactDetails":{ "mobile":"%s"%userId,"email":"%s@example.com"%userId,
               "address":{"street":"abcde","state":"Karnataka","pincode":"560001"}},
        "category": "Primary",
        "schoolType": "Co-ed"
    }

In [10]:
def getEntityPayload(userId,password):
    return {
      "client_id": "registry-frontend",
      "username": "%s"%userId,
      "password": "%s"%password,
      "grant_type": "password"
    }


In [11]:
getEntityPayload(123,"424fdcsa")

{'client_id': 'registry-frontend',
 'username': '123',
 'password': '424fdcsa',
 'grant_type': 'password'}

In [12]:
entityForStudent(2345)

{'identityDetails': {'fullName': 'Student 2345', 'gender': 'Male'},
 'ContactDetails': {'mobile': '2345',
  'email': '2345@example.com',
  'address': {'street': 'abc', 'state': 'Karnataka', 'pincode': '560001'}},
 'educationDetails': [{'institute': 'Some school'},
  {'institute': 'Some other school'}]}

In [13]:
entityForInstitute(2132157)

{'instituteName': '2132157',
 'ContactDetails': {'mobile': '2132157',
  'email': '2132157@example.com',
  'address': {'street': 'abcde', 'state': 'Karnataka', 'pincode': '560001'}},
 'category': 'Primary',
 'schoolType': 'Co-ed'}

In [14]:
entityForTeacher(12344)

{'personalDetails': {'userName': 12344, 'dob': '2020-12-12'},
 'IdentityDetails': {'type': 'id', 'value': 12344},
 'ContactDetails': {'mobile': '12344',
  'email': '12344@example.com',
  'address': {'street': 'abc', 'state': 'Karnataka', 'pincode': '560001'}}}

In [15]:
defaultPassowrd = "abcd@123"

# Invite Institution

In [16]:
instituteId =str(random.randint(1e10,1e11))
instituteId

'99726221245'

In [17]:
resp = requests.post("%s%s"%(base_url, instituteInvite), json=entityForInstitute(instituteId)
)
print(resp.content)
assert resp.status_code == 200
print(resp.json())
assert(resp.json()["id"]=="sunbird-rc.registry.invite")
assert resp.json()["params"]["status"] == "SUCCESSFUL"
entity_name=next(iter(resp.json()["result"].keys()))
assert "Institute"==entity_name
idx = resp.json()["result"][entity_name]["osid"]

b'{"id":"sunbird-rc.registry.invite","ver":"1.0","ets":1645765571833,"params":{"resmsgid":"","msgid":"901e614e-8933-48f9-9b9b-21aedc58f06d","err":"","status":"SUCCESSFUL","errmsg":""},"responseCode":"OK","result":{"Institute":{"osid":"1-3d8385cf-0b81-48e8-9983-182c908fc571"}}}'
{'id': 'sunbird-rc.registry.invite', 'ver': '1.0', 'ets': 1645765571833, 'params': {'resmsgid': '', 'msgid': '901e614e-8933-48f9-9b9b-21aedc58f06d', 'err': '', 'status': 'SUCCESSFUL', 'errmsg': ''}, 'responseCode': 'OK', 'result': {'Institute': {'osid': '1-3d8385cf-0b81-48e8-9983-182c908fc571'}}}


### Set Admin Role for Institution User

- Go to Key Cloack Admin Console
- Navigate to Users section and find the Institution User with above mentioned Institution Id
- Select Role mapping Tab and Add the Role edu-admin to the user

## Get Token for Institution 

In [18]:
getEntityPayload(instituteId,defaultPassowrd)

{'client_id': 'registry-frontend',
 'username': '99726221245',
 'password': 'abcd@123',
 'grant_type': 'password'}

In [19]:
headers = {
    'content-type': 'application/x-www-form-urlencoded',
}
data = getEntityPayload(instituteId,defaultPassowrd)
response = requests.post('http://keycloak:8080/auth/realms/sunbird-rc/protocol/openid-connect/token', headers=headers, data=data)
institutionToken = response.json()["access_token"]
institutionToken

'eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJsNGdqQkFjczdDd3BZdE9FUlE4SEQxcUNCYXNrY3BxN2tHcmk1WkFjTWJjIn0.eyJleHAiOjE2NDU3NjYzNTQsImlhdCI6MTY0NTc2NTc1NCwianRpIjoiNDdhZTdkNjYtNWU5ZC00MDBmLWIwMTItNzc1MzFiNDRmODdkIiwiaXNzIjoiaHR0cDovL2tleWNsb2FrOjgwODAvYXV0aC9yZWFsbXMvc3VuYmlyZC1yYyIsImF1ZCI6ImFjY291bnQiLCJzdWIiOiI0Y2Y2ZDQ3YS0yYjc3LTQ1ZTYtOGQ5YS02ZDNjMGMwYWEyNGQiLCJ0eXAiOiJCZWFyZXIiLCJhenAiOiJyZWdpc3RyeS1mcm9udGVuZCIsInNlc3Npb25fc3RhdGUiOiI1MjNkZTNjMy0zNTJhLTRhZjUtOGEwZC1hYTZhNDZmMDE1MjkiLCJhY3IiOiIxIiwiYWxsb3dlZC1vcmlnaW5zIjpbImh0dHBzOi8vbG9jYWxob3N0OjQyMDIiLCJodHRwOi8vbG9jYWxob3N0OjQyMDIiLCJodHRwczovL2xvY2FsaG9zdDo0MjAwIiwiaHR0cHM6Ly9uZGVhci54aXYuaW4iLCJodHRwOi8vbG9jYWxob3N0OjQyMDAiLCJodHRwOi8vbmRlYXIueGl2LmluIiwiaHR0cDovLzIwLjE5OC42NC4xMjgiXSwicmVhbG1fYWNjZXNzIjp7InJvbGVzIjpbImVkdS1hZG1pbiIsIm9mZmxpbmVfYWNjZXNzIiwidW1hX2F1dGhvcml6YXRpb24iLCJkZWZhdWx0LXJvbGVzLW5kZWFyIl19LCJyZXNvdXJjZV9hY2Nlc3MiOnsiYWNjb3VudCI6eyJyb2xlcyI6WyJtYW5hZ2UtYWNjb3VudCIsIm1hbmFnZS1hY2NvdW50LWxpbmtzIiwidml

# Invite Teacher with Token

In [20]:
teacherId =str(random.randint(1e10,1e11))
teacherId

'27790292520'

## Inviting with Invalid Token should give 401 Unautorized Exception

In [22]:
headers = {
    'content-type': 'application/json',
    'authorization': 'bearer %s'%institutionToken+"someR$andom$^$Inv%alidT@$&ext",
}
resp = requests.post("%s%s"%(base_url, teacherInvite), json=entityForTeacher(teacherId),headers= headers)
assert resp.status_code == 401

## Inviting with Valid Token should create Teacher Entity

In [23]:
headers = {
    'content-type': 'application/json',
    'authorization': 'bearer %s'%institutionToken,
}
resp = requests.post("%s%s"%(base_url, teacherInvite), json=entityForTeacher(teacherId),headers= headers)
print(resp.content)
assert resp.status_code == 200
print(resp.json())
assert(resp.json()["id"]=="sunbird-rc.registry.invite")
assert resp.json()["params"]["status"] == "SUCCESSFUL"
entity_name=next(iter(resp.json()["result"].keys()))
assert "Teacher"==entity_name
idx = resp.json()["result"][entity_name]["osid"]

b'{"id":"sunbird-rc.registry.invite","ver":"1.0","ets":1645765847524,"params":{"resmsgid":"","msgid":"050f5cc5-8f34-4ef8-b274-888137e74478","err":"","status":"SUCCESSFUL","errmsg":""},"responseCode":"OK","result":{"Teacher":{"osid":"1-911955c7-ffe4-419a-9d55-2cf076f91798"}}}'
{'id': 'sunbird-rc.registry.invite', 'ver': '1.0', 'ets': 1645765847524, 'params': {'resmsgid': '', 'msgid': '050f5cc5-8f34-4ef8-b274-888137e74478', 'err': '', 'status': 'SUCCESSFUL', 'errmsg': ''}, 'responseCode': 'OK', 'result': {'Teacher': {'osid': '1-911955c7-ffe4-419a-9d55-2cf076f91798'}}}


## Set Staff Role for Teacher User
- Go to Key Cloack Admin Console
- Navigate to Users section and find the Teacher User with above mentioned Teacher Id
- Select Role mapping Tab and Add the Role edu-staff to the user

## Get Token for Teacher

In [25]:
headers = {
    'content-type': 'application/x-www-form-urlencoded',
}
data = getEntityPayload(teacherId,defaultPassowrd)
response = requests.post('http://keycloak:8080/auth/realms/sunbird-rc/protocol/openid-connect/token', headers=headers, data=data)
teacherToken = response.json()["access_token"]

In [26]:
teacherToken

'eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJsNGdqQkFjczdDd3BZdE9FUlE4SEQxcUNCYXNrY3BxN2tHcmk1WkFjTWJjIn0.eyJleHAiOjE2NDU3NjY0OTYsImlhdCI6MTY0NTc2NTg5NiwianRpIjoiOTVhNDE0ZDAtMDEzOC00OGVkLTgwMTItMjE2MzY5NDRiN2I4IiwiaXNzIjoiaHR0cDovL2tleWNsb2FrOjgwODAvYXV0aC9yZWFsbXMvc3VuYmlyZC1yYyIsImF1ZCI6ImFjY291bnQiLCJzdWIiOiI0OTVlN2RkNy0yZGVjLTRiYTktYmNlZi01NzgzNjAyMjg2NjgiLCJ0eXAiOiJCZWFyZXIiLCJhenAiOiJyZWdpc3RyeS1mcm9udGVuZCIsInNlc3Npb25fc3RhdGUiOiI4ODJlMGYwMS00NjAyLTQyNjAtOTlhNy0wZDIyNmYzZTUwOGUiLCJhY3IiOiIxIiwiYWxsb3dlZC1vcmlnaW5zIjpbImh0dHBzOi8vbG9jYWxob3N0OjQyMDIiLCJodHRwOi8vbG9jYWxob3N0OjQyMDIiLCJodHRwczovL2xvY2FsaG9zdDo0MjAwIiwiaHR0cHM6Ly9uZGVhci54aXYuaW4iLCJodHRwOi8vbG9jYWxob3N0OjQyMDAiLCJodHRwOi8vbmRlYXIueGl2LmluIiwiaHR0cDovLzIwLjE5OC42NC4xMjgiXSwicmVhbG1fYWNjZXNzIjp7InJvbGVzIjpbIm9mZmxpbmVfYWNjZXNzIiwiZWR1LXN0YWZmIiwidW1hX2F1dGhvcml6YXRpb24iLCJkZWZhdWx0LXJvbGVzLW5kZWFyIl19LCJyZXNvdXJjZV9hY2Nlc3MiOnsiYWNjb3VudCI6eyJyb2xlcyI6WyJtYW5hZ2UtYWNjb3VudCIsIm1hbmFnZS1hY2NvdW50LWxpbmtzIiwidml

## Invite a new Student

In [27]:
studentId =str(random.randint(1e10,1e11))
studentId

'73653668481'

### Anonymously

In [28]:
headers = {
    'content-type': 'application/json'
}
resp = requests.post("%s%s"%(base_url, studentInvite), json=entityForStudent(studentId),headers= headers)
assert resp.json()['params']['status'] == 'UNSUCCESSFUL'
assert resp.status_code == 401
print(resp.json())

AssertionError: 

### With Institution Token

In [30]:
headers = {
    'content-type': 'application/json',
    'authorization': 'bearer %s'%institutionToken,
}
data = entityForStudent(studentId)
resp = requests.post("%s%s"%(base_url, studentInvite), json=data,headers=headers)
assert resp.status_code == 200
print(resp)

<Response [200]>


In [31]:
assert resp.status_code == 200
print(resp.json())
assert(resp.json()["id"]=="sunbird-rc.registry.invite")
assert resp.json()["params"]["status"] == "SUCCESSFUL"
entity_name=next(iter(resp.json()["result"].keys()))
assert "Student"==entity_name
idx = resp.json()["result"][entity_name]["osid"]

{'id': 'sunbird-rc.registry.invite', 'ver': '1.0', 'ets': 1645766034540, 'params': {'resmsgid': '', 'msgid': '5f44ee61-7bb4-4773-9690-43a689f7978f', 'err': '', 'status': 'SUCCESSFUL', 'errmsg': ''}, 'responseCode': 'OK', 'result': {'Student': {'osid': '1-58f87294-f325-404a-9958-82722225d132'}}}


### With Teacher Token

In [32]:
studentId2 =str(random.randint(1e10,1e11))
studentId2

'93493876537'

In [33]:
headers = {
    'content-type': 'application/json',
    'authorization': 'bearer %s'%teacherToken,
}
resp = requests.post("%s%s"%(base_url, studentInvite), json=entityForStudent(studentId2),headers=headers)
print(resp)
assert resp.status_code == 200

<Response [200]>


In [34]:
assert(resp.json()["id"]=="sunbird-rc.registry.invite")
assert resp.json()["params"]["status"] == "SUCCESSFUL"
entity_name=next(iter(resp.json()["result"].keys()))
assert "Student"==entity_name
idx = resp.json()["result"][entity_name]["osid"]

### With Student Token

In [35]:
studentId3 =str(random.randint(1e10,1e11))
studentId3

'15120976649'

In [38]:
headers = {
    'content-type': 'application/x-www-form-urlencoded',
}
data = getEntityPayload(studentId,defaultPassowrd)
response = requests.post('http://keycloak:8080/auth/realms/sunbird-rc/protocol/openid-connect/token', headers=headers, data=data)
studentToken = response.json()["access_token"]

In [39]:
headers = {
    'content-type': 'application/json',
    'authorization': 'bearer %s'%studentToken,
}
resp = requests.post("%s%s"%(base_url, studentInvite), json=entityForStudent(studentId3),headers=headers)
print(resp)
assert resp.status_code == 401

<Response [200]>


AssertionError: 